# Two Moon - Distance Manifold - Hug (No Tangential)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, cos, sin
from numpy.linalg import svd, inv
from scipy.stats import multivariate_normal
from numpy.random import uniform, normal
from numpy.linalg import norm
from scipy.optimize import fsolve

# Helper Functions

In [3]:
def proj(A):
    """Computes projection matrix onto space spanned by columns of A"""
    return A @ (inv(A.T @ A) @ A.T)

# Two Moon Simulator

In [5]:
def TM_simulator(xi):
    """Two Moons deterministic simulator for ABC."""
    t0, t1, a, r = xi[0], xi[1], xi[2], xi[3]
    p = np.array([r * np.cos(a) + 0.25, r * np.sin(a)])
    return p + np.array([-np.abs(t0 + t1), (-t0 + t1)]) / sqrt(2)

In [37]:
def gradient(xi):
    """Computes Q. This is the transpose of the Jacobian."""
    t1, t2, a, r = xi
    val = - (t1 + t2) / (abs(t1 + t2) * sqrt(2))
    return np.array([
        [val, val, -r * sin(a), cos(a)],
        [-1/sqrt(2), 1/sqrt(2), r * cos(a), sin(a)]
    ]).T

def tangent(g):
    """Computes tangent given a gradient g"""
    return svd(g)[0][:, 2:]

def tcov(lam, mu, t):
    """Computes tangential covariance"""
    return ((mu**2)*np.eye(4) + (lam**2 - mu**2)*proj(t)) / np.linalg.norm(t)**2

# Observed Data

In [38]:
y_star = np.zeros(2)

# Initial Point (on Manifold)

In [39]:
# Sample randomly the seeds
a0 = uniform(low=-np.pi/2, high=np.pi/2)
r0 = normal(loc=0.1, scale=0.01)
# Start with a guess
guess = np.array([0, 0, a0, r0])
# Find point on manifold using optimization
func = lambda xi: np.r_[TM_simulator(xi) - y_star, 0, 0]  # Append 0, 0 to make fsolve work.
xi0 = fsolve(func, guess)

# Tangential Hug Settings

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678],
       [-0.11189476,  0.00697233],
       [ 0.06219092,  0.99806427]])

# Run THug (no function)

In [19]:
# Compute gradient basis at xi0
g0 = gradient(xi0)
# Compute tangent basis at xi0 
t0 = tangent(g0)
# Instantiate tangential MVN
#multivariate_normal()

In [23]:
t0

array([[-0.08352208,  0.52867321],
       [-0.07485918, -0.46940157],
       [ 0.9936041 ,  0.01838085],
       [-0.01307865,  0.70698582]])

In [30]:
t0 @ (inv(t0.T @ t0) @ t0.T)

(4, 4)

In [33]:
proj(t0)

array([[ 2.86471300e-01, -2.41907640e-01, -7.32704168e-02,
         3.74856818e-01],
       [-2.41907640e-01,  2.25941730e-01, -8.30083899e-02,
        -3.30881196e-01],
       [-7.32704168e-02, -8.30083899e-02,  9.87586970e-01,
         1.21430643e-17],
       [ 3.74856818e-01, -3.30881196e-01,  1.04083409e-17,
         5.00000000e-01]])

# Display


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# Prettify

# Check if two versions are equivalent